# FAST-UAV - Multirotor Design Optimization

[FAST-OAD](https://fast-oad.readthedocs.io) is a framework for performing rapid Overall Aircraft Design. The computational core of FAST-OAD is based on the  [OpenMDAO framework](https://openmdao.org/). <br>
FAST-UAV is the drone declination of FAST-OAD.

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the [DJI Matrice 600 Pro](https://www.dji.com/matrice600-pro) multicopter. In `workdir/`, we store files generated or modified by FAST-UAV.

In [1]:
import os.path as pth
import openmdao.api as om
import logging
import shutil
import fastoad.api as oad
from time import time
import matplotlib.pyplot as plt
from fastuav.utils.postprocessing.analysis_and_plots import *

plt.rcParams["figure.figsize"] = 16, 8
plt.rcParams.update({"font.size": 13})

DATA_FOLDER_PATH = "../data"
CONFIGURATIONS_FOLDER_PATH = "../configurations"
WORK_FOLDER_PATH = "./workdir"

CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, "multirotor_mdo.yaml")
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, "problem_inputs_DJI_M600.xml")

# For having log messages display on screen
logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

# For using all screen width
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
oad.generate_configuration_file(
    CONFIGURATION_FILE, overwrite=True, distribution_name="fastuav", sample_file_name="multirotor_mdo.yaml"
)

INFO    : Loading FAST-OAD plugin bundled
INFO    : Loading bundles from fastoad.models
INFO    : Installed bundle fastoad.models.performances.mission.openmdao.mission (ID 18 )
INFO    : Installed bundle fastoad.models.__init__ (ID 35 )
INFO    : Installed bundle fastoad.models.performances.mission.exceptions (ID 11 )
INFO    : Installed bundle fastoad.models.performances.mission.routes (ID 23 )
INFO    : Installed bundle fastoad.models.performances.mission.segments.hold (ID 27 )
INFO    : Installed bundle fastoad.models.performances.mission.openmdao.mission_wrapper (ID 19 )
INFO    : Installed bundle fastoad.models.performances.__init__ (ID 34 )
INFO    : Installed bundle fastoad.models.performances.mission.mission_definition.resources.__init__ (ID 14 )
INFO    : Installed bundle fastoad.models.performances.mission.segments.cruise (ID 26 )
INFO    : Installed bundle fastoad.models.performances.mission.mission_definition.mission_builder (ID 13 )
INFO    : Installed bundle fastoad.model

INFO    : Installed bundle fastuav.models.stability.static_longitudinal.__init__ (ID 125 )
INFO    : Installed bundle fastuav.models.performance.mission.mission_definition.schema (ID 57 )
INFO    : Installed bundle fastuav.models.propulsion.gearbox.__init__ (ID 82 )
INFO    : Installed bundle fastuav.models.geometry.geometry_hybrid (ID 46 )
INFO    : Installed bundle fastuav.models.mtow.mtow_hybrid (ID 51 )
INFO    : Installed bundle fastuav.models.propulsion.propeller.propeller (ID 97 )
INFO    : Installed bundle fastuav.models.add_ons.__init__ (ID 37 )
INFO    : Installed bundle fastuav.models.propulsion.propeller.performance_analysis (ID 96 )
INFO    : Installed bundle fastuav.models.structures.fuselage (ID 127 )
INFO    : Installed bundle fastuav.models.propulsion.propeller.__init__ (ID 98 )
INFO    : Installed bundle fastuav.models.__init__ (ID 143 )
INFO    : Installed bundle fastuav.models.geometry.geometry_multirotor (ID 47 )
INFO    : Installed bundle fastuav.models.geometry._

'C:\\Users\\User\\Documents\\GitHub\\FAST-UAV\\src\\fastuav\\notebooks\\workdir\\multirotor_mdo.yaml'

The YAML configuration file located in the data folder defines the design problem, i.e. the model, the problem driver and the optimization problem definition.<br>
A useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [3]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width="100%", height="500px")

INFO    : N2 diagram written in C:\Users\User\Documents\GitHub\FAST-UAV\src\fastuav\notebooks\workdir\n2.html


In the configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-UAV to generate the inputs of the model with the reference parameters from 'problem_inputs_DJI_M600.xml' as default values:

In [4]:
oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Problem inputs written in C:\Users\User\Documents\GitHub\FAST-UAV\src\fastuav\notebooks\workdir\problem_inputs.xml


'C:\\Users\\User\\Documents\\GitHub\\FAST-UAV\\src\\fastuav\\notebooks\\workdir\\problem_inputs.xml'

You can now checkout the generated [input file](./workdir/problem_inputs.xml). The values in this file can be modified by the user and will be considered by FAST-UAV when executing a computational process.<br>
The `variable-viewer` provides a way to inspect and modify the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [6]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_inputs.xml")
oad.variable_viewer(INPUT_FILE)

C:\Users\User\AppData\Roaming\Python\Python38\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


## 2. Running an MDO

You can now run an optimization problem. The last part of the configuration file .yaml is where this optimization problem is defined:

```yaml
optimization:
  design_variables:
    - name: data:weights:mtow:k # over estimation coefficient on the load mass
      upper: 40.0
      lower: 1.0
  constraints:
    - name: data:weights:mtow:guess:constraint # mass consistency
      lower: 0.0
  objective:
    - name: data:weights:mtow
      scaler: 1e-1
```

In [44]:
optim_problem = oad.optimize_problem(CONFIGURATION_FILE, overwrite=True)

Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: [2.57212067]
            Iterations: 61
            Function evaluations: 443
            Gradient evaluations: 57
Optimization FAILED.
Positive directional derivative for linesearch
-----------------------------------


ERROR   : Optimization failed after 11.83 seconds
INFO    : Problem outputs written in C:\Users\User\Documents\GitHub\FAST-UAV\src\fastuav\notebooks\workdir\problem_outputs.xml


Let's save these results:

In [45]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "problem_outputs.xml")
DJI_M600_OUTPUT_FILE = pth.join(DATA_FOLDER_PATH, 'problem_outputs_DJI_M600_mdo.xml')
shutil.copy(OUTPUT_FILE, DJI_M600_OUTPUT_FILE)

'../data\\problem_outputs_DJI_M600_mdo.xml'

The `optimizer_viewer` offers a convenient summary of the optimization result. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated.

In [46]:
oad.optimization_viewer(CONFIGURATION_FILE)

C:\Users\User\AppData\Roaming\Python\Python38\site-packages\jupyter_client\session.py:716: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [47]:
oad.variable_viewer(OUTPUT_FILE)

## 3. Analysis and plots

You can now use postprocessing plots to visualize the results of the MDO.

In [48]:
fig = multirotor_geometry_plot(DJI_M600_OUTPUT_FILE, name="Drone MDO")
fig.show()

In [49]:
fig = mass_breakdown_sun_plot_drone(DJI_M600_OUTPUT_FILE)
fig.show()